<a href="https://colab.research.google.com/github/gmarianini/TFG_ECG_Arrhythmia_Detection/blob/main/Proyecto_ECG_Raspberry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitor ECG en Raspberry Pi con IA

Este cuaderno contiene el Firmware de producción diseñado para la detección temprana de arritmias cardíacas operando bajo el paradigma de Edge AI. El sistema está optimizado para ejecutarse de forma autónoma en una Raspberry Pi 4 Model B, eliminando la dependencia de conectividad a internet y garantizando la privacidad de los datos sensibles del paciente.

  Arquitectura del Sistema de Producción

El software orquesta un pipeline de procesamiento de baja latencia estructurado en tres capas jerárquicas:

*   Capa de Adquisición (Bus SPI): Controlador dedicado para el chip AD7920, garantizando una tasa de muestreo estricta de 360 Hz.
*   Capa de DSP y Sincronización: * Filtrado Butterworth de 4º orden (0.5-100 Hz) y Filtro Notch (50 Hz) para mitigar el ruido electromagnético y la deriva de línea base.

* Lógica "Look-Ahead": El sistema retiene el procesamiento hasta completar la ventana de 600 ms tras el pico R, asegurando que la red analice el complejo P-QRS-T completo.

* Capa de Inferencia (TFLite): Motor de ejecución del modelo MLP híbrido que procesa simultáneamente morfología y ritmo (Intervalo RR).

📊 Especificaciones Técnicas y Rendimiento

* Hardware Requerido: Raspberry Pi 4 (CPU ARM Cortex-A72), Sensor AD8232 y ADC AD7920.

* Latencia de Inferencia: 0.1 ms (tiempo de ejecución del modelo).

*  Latencia Visual Total: ~300.1 ms (incluyendo la ventana de seguridad fisiológica).

* Consumo de Recursos: Optimizado para operación en "Solo CPU" con una huella de memoria mínima.

⚠️ Protocolo de Seguridad Clínica

El firmware incluye un mecanismo de triaje automático: si la densidad de arritmias supera el 10% en la ventana actual, el sistema activa una Parada de Emergencia, congela el trazado para revisión médica y despliega una alerta visual de alto contraste.

In [ ]:
import spidev

class AD7920:
    def __init__(self, bus=0, device=0):
        """
        Constructor del driver para el ADC AD7920.
        Se comunica por SPI.
        """
        try:
            self.spi = spidev.SpiDev()
            self.spi.open(bus, device)

            # Velocidad: 16 MHz (El chip aguanta hasta 20 MHz)
            self.spi.max_speed_hz = 16000000

            # Modo 0 (CPOL=0, CPHA=0) es el estándar de este chip
            self.spi.mode = 0b00
        except Exception as e:
            print(f"ERROR CRÍTICO SPI: No se pudo abrir el bus SPI. {e}")

    def read_adc(self):
        """
        Lee el valor actual del sensor (0 a 4095).
        """
        # Enviamos 2 bytes vacíos para recibir 2 bytes de datos
        # El chip funciona enviando reloj y devolviendo datos simultáneamente
        lectura = self.spi.xfer2([0x00, 0x00])

        if not lectura:
            return 0

        # --- RECONSTRUCCIÓN MATEMÁTICA ---
        # Byte Alto (MSB) + Byte Bajo (LSB)
        valor_crudo = (lectura[0] << 8) + lectura[1]

        # Desplazamos 1 bit a la derecha para alinear (según datasheet)
        valor_ajustado = (valor_crudo >> 1)

        # Limpiamos con máscara de 12 bits (4095)
        return valor_ajustado & 0xFFF

    def close(self):
        """Cierra la conexión SPI"""
        self.spi.close()

In [ ]:
import sys
import types
import importlib.util
import collections
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.signal import iirnotch, lfilter, butter, sosfilt, find_peaks

# =========================================================
# PARCHE DE EMERGENCIA (Python 3.13 + TensorFlow)
# =========================================================
if "imp" not in sys.modules:
    fake_imp = types.ModuleType("imp")
    def find_module(name, path=None): return None, None, None
    fake_imp.reload = importlib.reload
    fake_imp.find_module = find_module
    sys.modules["imp"] = fake_imp
# =========================================================

import tensorflow as tf
from ad7920_driver import AD7920

# =========================================================
# CONFIGURACIÓN CRÍTICA
# =========================================================
MODEL_PATH = 'modelo_ecg_edge.tflite'
INPUT_SIGNAL_LEN = 216
TOTAL_INPUTS = 217

# --- AJUSTE DE SENSIBILIDAD ---
# En la validación usaba 0.85 (muy estricto), bajamos el umbral para el sistema en tiempo real
UMBRAL_ARRITMIA = 0.50

# Variables globales
contador_arritmias = 0
latencia_ultima = 0.0
prob_arritmia_actual = 0.0

# --- CARGA DEL MODELO TFLITE ---
interpreter = None
try:
    print(f"--- CARGANDO IA (TFLITE) ---")
    interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(f" MODELO CARGADO (Umbral: {UMBRAL_ARRITMIA})")
except Exception as e:
    print(f" ERROR IA: {e}")

# =========================================================
# CONFIGURACIÓN SEÑAL
# =========================================================
FS = 360.0
SECONDS_VIEW = 4.0
WINDOW_LEN = int(FS * SECONDS_VIEW)

# Filtros Médicos
b_notch, a_notch = iirnotch(50.0, 30.0, FS)
sos_bandpass = butter(4, [0.5, 100.0], btype='band', fs=FS, output='sos')

adc = AD7920()

# Buffers
MAX_BUFFER = WINDOW_LEN + 1000
buffer_raw = collections.deque(maxlen=MAX_BUFFER)
buffer_clean_full = collections.deque(maxlen=MAX_BUFFER)

for _ in range(MAX_BUFFER):
    buffer_raw.append(2048)
    buffer_clean_full.append(0)

# =========================================================
# INTERFAZ GRÁFICA
# =========================================================
fig, ax = plt.subplots(figsize=(12, 6))
line, = ax.plot([], [], lw=1.2, color='#00ff00', label='ECG')
peaks_plot, = ax.plot([], [], 'ro', markersize=6)

ax.set_facecolor('black')
fig.patch.set_facecolor('#101010')
ax.set_ylim(-2000, 2000)
ax.set_xlim(0, WINDOW_LEN)
ax.grid(True, color='#333333', linestyle='--', linewidth=0.5)
ax.set_title(f"ECG MONITOR - HIGH SENSITIVITY MODE", color='white')

# Textos HUD
texto_pred = ax.text(0.5, 0.88, 'WAITING...', transform=ax.transAxes,
                     fontsize=14, fontweight='bold', color='gray', ha='center',
                     bbox=dict(facecolor='black', edgecolor='gray', alpha=0.8))

texto_bpm = ax.text(0.92, 0.88, '-- BPM', transform=ax.transAxes, fontsize=16,
                    fontweight='bold', color='#00ff00', va='center', ha='center',
                    bbox=dict(facecolor='black', edgecolor='white', boxstyle='round,pad=0.3'))

texto_metrics = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=10,
                        fontweight='bold', family='monospace', color='white', va='top',
                        bbox=dict(facecolor='black', edgecolor='#333333', alpha=0.6))

# Control
total_samples_seen = 0
last_processed_absolute_idx = 0
prev_peak_absolute_idx = 0

# =========================================================
# LÓGICA IA
# =========================================================
def predict_heartbeat(segmento, rr_seconds):
    global latencia_ultima, prob_arritmia_actual

    if interpreter is None: return "IA OFF", False

    # 1. Normalización (IGUAL QUE EN VALIDACIÓN)
    # segment_norm = (segment - np.mean(segment)) / np.std(segment)
    std_val = np.std(segmento)
    if std_val < 0.001: return "RUIDO", False
    segmento_norm = (segmento - np.mean(segmento)) / std_val

    # 2. Construcción del Input (217 valores)
    # input_vector = np.append(segment_norm, rr_seconds)
    input_vector = np.append(segmento_norm, rr_seconds).astype(np.float32)
    input_data = input_vector.reshape(1, TOTAL_INPUTS)

    # 3. Inferencia
    t_start = time.time()
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Obtenemos la probabilidad cruda (0.0 a 1.0)
    probabilidad = interpreter.get_tensor(output_details[0]['index'])[0][0]
    t_end = time.time()

    # Guardamos métricas
    latencia_ultima = (t_end - t_start) * 1000
    prob_arritmia_actual = probabilidad * 100

    # Debug en consola
    print(f"❤️ RR: {rr_seconds:.2f}s | 🤖 PROB ARRITMIA: {probabilidad:.4f}")

    # 4. Decisión basada en Umbral
    es_arritmia = False
    texto_salida = "NORMAL"

    if probabilidad > UMBRAL_ARRITMIA:
        es_arritmia = True
        texto_salida = f"ARRITMIA ({int(prob_arritmia_actual)}%)"
    else:
        es_arritmia = False
        texto_salida = f"NORMAL ({int(100 - prob_arritmia_actual)}%)"

    return texto_salida, es_arritmia

# =========================================================
# BUCLE
# =========================================================
def update(frame):
    global total_samples_seen, last_processed_absolute_idx, prev_peak_absolute_idx, contador_arritmias

    # Adquisición
    nuevos = []
    for _ in range(12): nuevos.append(adc.read_adc())
    buffer_raw.extend(nuevos)
    total_samples_seen += 12

    # Filtrado
    raw_np = np.array(buffer_raw)
    notch = lfilter(b_notch, a_notch, raw_np)
    clean = sosfilt(sos_bandpass, notch)

    datos_limpios_nuevos = clean[-12:]
    buffer_clean_full.extend(datos_limpios_nuevos)

    clean_full_np = np.array(buffer_clean_full)
    datos_visibles = clean_full_np[-WINDOW_LEN:]

    # Detección
    umbral = np.max(datos_visibles) * 0.5
    if umbral < 400: umbral = 400
    indices_picos_visibles, _ = find_peaks(datos_visibles, distance=144, height=umbral)

    # Lógica de Gatillo
    if len(indices_picos_visibles) > 0:
        pico_relativo = indices_picos_visibles[-1]
        pico_absoluto = total_samples_seen - (WINDOW_LEN - pico_relativo)
        mitad = INPUT_SIGNAL_LEN // 2

        samples_future = total_samples_seen - pico_absoluto

        if pico_absoluto > last_processed_absolute_idx and samples_future >= mitad:

            # --- CÁLCULO DE RR (IGUAL QUE EN VALIDACIÓN) ---
            rr_sec = 0.85 # Default primer latido

            if prev_peak_absolute_idx > 0:
                diff = pico_absoluto - prev_peak_absolute_idx
                rr_sec = diff / FS

            # IMPORTANTE: Aplicamos el mismo límite que en el entrenamiento
            # if rr_seconds > 2.5: rr_seconds = 2.5
            if rr_sec > 2.5: rr_sec = 2.5
            if rr_sec < 0.2: rr_sec = 0.85 # Protección contra ruido

            prev_peak_absolute_idx = last_processed_absolute_idx
            last_processed_absolute_idx = pico_absoluto

            # Recortar ventana
            idx_buffer = len(clean_full_np) - (total_samples_seen - pico_absoluto)
            inicio = idx_buffer - mitad
            fin = idx_buffer + mitad

            if inicio >= 0 and fin <= len(clean_full_np):
                segmento = clean_full_np[inicio:fin]
                if len(segmento) == INPUT_SIGNAL_LEN:

                    texto, es_arritmia = predict_heartbeat(segmento, rr_sec)

                    # UI
                    color_txt = '#00ff00'
                    borde = 'white'

                    if es_arritmia:
                        color_txt = 'red'
                        borde = 'red'
                        contador_arritmias += 1

                    texto_pred.set_text(texto)
                    texto_pred.set_color(color_txt)
                    texto_pred.set_bbox(dict(facecolor='black', edgecolor=borde))

                    # BPM
                    bpm = 60.0 / rr_sec
                    if 30 < bpm < 220:
                        texto_bpm.set_text(f"{int(bpm)} BPM")

    # Gráfica
    line.set_data(range(len(datos_visibles)), datos_visibles)
    if len(indices_picos_visibles) > 0:
        peaks_plot.set_data(indices_picos_visibles, datos_visibles[indices_picos_visibles])

    raw_val = np.mean(nuevos)
    estado = "OK"
    if raw_val > 4050 or raw_val < 50:
        estado = "LEADS OFF"
        line.set_color("red")
    else:
        line.set_color("#00ff00")

    texto_metrics.set_text(
        f"SENSOR   : {estado}\n"
        f"UMBRAL   : {UMBRAL_ARRITMIA}\n" # Muestro el umbral
        f"PROB     : {prob_arritmia_actual:.1f} %\n"
        f"COUNT    : {contador_arritmias}\n"
    )

    return line, peaks_plot, texto_pred, texto_bpm, texto_metrics

ani = FuncAnimation(fig, update, interval=20, blit=True, cache_frame_data=False)
plt.show()